In [5]:
import pandas as pd

file_path = "ab_test_log_with_times2.csv"
df = pd.read_csv(file_path)

# Replace 'user_id' with a new 'user' column numbered 
df['user'] = range(1, 201)

# Drop the original 'user_id' column if it exists
if 'user_id' in df.columns:
    df = df.drop(columns=['user_id'])
    
cols = ['user'] + [col for col in df.columns if col != 'user']
df = df[cols]

df['session_start_time'] = pd.to_datetime(df['session_start_time'])
df['session_end_time'] = pd.to_datetime(df['session_end_time'])

# 1. total_session_time in seconds
df['total_session_time'] = (df['session_end_time'] - df['session_start_time']).dt.total_seconds()

# 2. total_clicked_count
df['total_clicked_count'] = (
    df['revert_button_clicked_count'].fillna(0) +
    df['download_button_clicked_count'].fillna(0) +
    df['apply_fe_button_clicked_count'].fillna(0)
)

# 3. total_error_count
df['total_error_count'] = (
    df['revert_button_error_count'].fillna(0) +
    df['apply_fe_button_error_count'].fillna(0) +
    df['download_button_error_count'].fillna(0)
)

# 4. Clicked and error rates
df['apply_fe_button_clicked_rate'] = df['apply_fe_button_clicked_count'] / df['total_session_time']
df['revert_button_clicked_rate'] = df['revert_button_clicked_count'] / df['total_session_time']
df['download_button_clicked_rate'] = df['download_button_clicked_count'] / df['total_session_time']
df['total_clicked_rate'] = df['total_clicked_count'] / df['total_session_time']
df['total_error_rate'] = df['total_error_count'] / df['total_session_time']

# 5. Count number of each type of error in the operation column
operation_cols = [col for col in df.columns if col.startswith("operation_name")]
error_keywords = ['normalize', 'one_hot', 'box_cox', 'convert_date']
error_column_names = [f'{err}_error_count' for err in error_keywords]

for keyword, col_name in zip(error_keywords, error_column_names):
    df[col_name] = df[operation_cols].apply(
        lambda row: row.astype(str).str.contains(keyword).sum(), axis=1
    )


print(df.head())

   user group         session_start_time           session_end_time  \
0     1   red 2025-04-18 11:38:34.899631 2025-04-18 11:40:50.559276   
1     2  gray 2025-04-16 00:30:52.955283 2025-04-16 00:35:44.309154   
2     3  gray 2025-04-19 11:31:41.608770 2025-04-19 11:36:17.920322   
3     4   red 2025-04-19 02:00:43.497523 2025-04-19 02:03:20.240522   
4     5   red 2025-04-18 07:57:57.241578 2025-04-18 08:00:26.418550   

   apply_fe_button_clicked_count  apply_fe_button_error_count  \
0                              0                            0   
1                              6                            2   
2                              6                            3   
3                              0                            0   
4                              7                            2   

   revert_button_clicked_count  download_button_clicked_count  \
0                            0                            0.0   
1                            2                      

In [6]:
## outliers

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from scipy.stats import zscore

# Select the columns for which outliers should be detected
cols_to_check = [
    'total_session_time',
    'apply_fe_button_clicked_count',
    'apply_fe_button_error_count',
    'revert_button_clicked_count',
    'download_button_clicked_count'
]

# Method 1: Detecting Outliers with 3.0 IQR
# --------------------
# If variables like apply_fe_button_clicked_count or total_session_time are highly right-skewed 
# (mostly low values, a few oversized), even with 3 × IQR, all samples with long right tails will
# still be recognized as exceptions.
# --------------------
iqr_outliers = set()
for col in cols_to_check:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    iqr_outliers.update(outliers['user'])


# Thus, we use other methods to cross-validate

# Method 2: Detecting Outliers with Z-score
z_scores = df[cols_to_check].apply(zscore)
zscore_outliers = set(df[(z_scores.abs() > 3).any(axis=1)]['user'])

# Method 3: Detecting Outliers with DBSCAN
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[cols_to_check].fillna(0))
db = DBSCAN(eps=1.5, min_samples=5).fit(scaled_data)
dbscan_outliers = set(df[db.labels_ == -1]['user'])

# Identify users who are detected as abnormal by all three methods
common_outliers = iqr_outliers & zscore_outliers & dbscan_outliers

iqr_outliers_list = sorted(iqr_outliers)
zscore_outliers_list = sorted(zscore_outliers)
dbscan_outliers_list = sorted(dbscan_outliers)
common_outliers_list = sorted(common_outliers)

print(iqr_outliers_list)
common_outliers_list

[2, 3, 5, 7, 8, 13, 17, 18, 27, 29, 30, 35, 42, 43, 48, 49, 52, 56, 57, 61, 69, 71, 80, 97, 101, 106, 112, 119, 126, 129, 131, 137, 138, 143, 144, 145, 147, 153, 154, 156, 167, 168, 169, 171, 173, 186, 192, 193]


[7, 27, 29, 61, 71]

In [7]:
final_outliers = common_outliers_list

df_outliers = df[df['user'].isin(final_outliers)]

# Remove abnormal user data and retain normal users
df_cleaned = df[~df['user'].isin(final_outliers)]

cleaned_path = "ab_test_log_step3_cleaned.csv"
outliers_path = "ab_test_log_step3_outliers.csv"

df_cleaned.to_csv(cleaned_path, index=False)
df_outliers.to_csv(outliers_path, index=False)

